In [1]:
from torchvision import transforms
import torch
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import timeit
import pandas as pd
import gc

import utils.image as im
import utils.cam as cam
import utils.grabcut as gcut
import utils.voc as voc
import utils.json as json
from utils.VOCSegmentation import VOCSegmentation

In [2]:
root_path = r'C:\Users\Nicol\Documents\EPFL\BA7\Project\Code\voc\\'

json_path = r'C:\Users\Nicol\Documents\EPFL\BA7\Project\Code\voc\json\\'

output_path = r'C:\Users\Nicol\Documents\EPFL\BA7\Project\Code\voc\sessions\1 Object grabcut\\'

In [3]:
data_tbl = VOCSegmentation(root = root_path,
                           year = '2012',
                           image_set = 'trainval',
                           download = False,
                           transform = transforms.ToTensor(),
                           target_transform = transforms.ToTensor(),
                           transforms = None,
                           target = 'Object')

In [4]:
data = iter(torch.utils.data.DataLoader(data_tbl,
                                        batch_size = 1,
                                        shuffle = False,
                                        num_workers = 2))

In [5]:
annotations = json.open_json(json_path + "voc-object-annotations-clean")
N = len(annotations)
annotations = iter(annotations.items())

In [6]:
IoU = np.array([])
time = np.array([])

for i in range(N):
    gc.collect()
    
    start = timeit.default_timer()
    
    img, sgm = next(data)
    img, sgm = torch.squeeze(img), torch.squeeze(sgm)

    img_cv2 = im.pil_to_cv2(img.numpy())
    sgm     = im.f1_to_f255(sgm.numpy())
    
    name, annots = next(annotations)
    
    for j, annot in enumerate(annots):
        _, _, cbbox = annot
        
        _, _, pred = gcut.grabcut(img_cv2, im.corner_to_delta(cbbox), mode = 'RECT')
        
        true = voc.true_mask(sgm, j+1, dilation = 0)
        
        IoU = np.append(IoU, voc.IoU_acc_undef(true, pred, sgm))
        
    stop = timeit.default_timer()
    time = np.append(time, stop - start)
    if i % 20 == 0:
        print(f'Image nb {i}')
        print(f'Time spent               = ' + voc.time_str(np.sum(time)))
        print(f'Estimated time remaining = ' + voc.time_str(np.mean(time) * (N - 1 - i)))
        print(f'Mean IoU                 =  {np.mean(IoU) :.3f}')
        print()

Image nb 0
Time spent               =   0m00s
Estimated time remaining =  28m47s
Mean IoU                 =  0.445

Image nb 20
Time spent               =   0m17s
Estimated time remaining =  39m34s
Mean IoU                 =  0.512

Image nb 40
Time spent               =   0m38s
Estimated time remaining =  45m16s
Mean IoU                 =  0.473

Image nb 60
Time spent               =   1m01s
Estimated time remaining =  48m00s
Mean IoU                 =  0.458

Image nb 80
Time spent               =   1m21s
Estimated time remaining =  47m18s
Mean IoU                 =  0.489

Image nb 100
Time spent               =   1m40s
Estimated time remaining =  46m51s
Mean IoU                 =  0.476

Image nb 120
Time spent               =   2m02s
Estimated time remaining =  47m10s
Mean IoU                 =  0.497

Image nb 140
Time spent               =   2m18s
Estimated time remaining =  45m31s
Mean IoU                 =  0.506

Image nb 160
Time spent               =   2m35s
Estimated time

Image nb 1400
Time spent               =  20m31s
Estimated time remaining =  22m08s
Mean IoU                 =  0.525

Image nb 1420
Time spent               =  20m47s
Estimated time remaining =  21m49s
Mean IoU                 =  0.526

Image nb 1440
Time spent               =  21m02s
Estimated time remaining =  21m29s
Mean IoU                 =  0.526

Image nb 1460
Time spent               =  21m16s
Estimated time remaining =  21m08s
Mean IoU                 =  0.527

Image nb 1480
Time spent               =  21m28s
Estimated time remaining =  20m46s
Mean IoU                 =  0.528

Image nb 1500
Time spent               =  21m45s
Estimated time remaining =  20m27s
Mean IoU                 =  0.528

Image nb 1520
Time spent               =  21m57s
Estimated time remaining =  20m05s
Mean IoU                 =  0.529

Image nb 1540
Time spent               =  22m14s
Estimated time remaining =  19m47s
Mean IoU                 =  0.529

Image nb 1560
Time spent               =  22m30s

Image nb 2780
Time spent               =  41m13s
Estimated time remaining =   1m57s
Mean IoU                 =  0.527

Image nb 2800
Time spent               =  41m35s
Estimated time remaining =   1m39s
Mean IoU                 =  0.526

Image nb 2820
Time spent               =  41m59s
Estimated time remaining =   1m22s
Mean IoU                 =  0.526

Image nb 2840
Time spent               =  42m22s
Estimated time remaining =   1m04s
Mean IoU                 =  0.526

Image nb 2860
Time spent               =  42m47s
Estimated time remaining =   0m46s
Mean IoU                 =  0.524

Image nb 2880
Time spent               =  43m09s
Estimated time remaining =   0m28s
Mean IoU                 =  0.523

Image nb 2900
Time spent               =  43m37s
Estimated time remaining =   0m10s
Mean IoU                 =  0.521



In [7]:
print(f'Mean IoU = {np.mean(IoU)}')

Mean IoU = 0.5210917350877718


In [8]:
df = pd.read_csv(r'C:\Users\Nicol\Documents\EPFL\BA7\Project\Code\voc\json\voc-object-annotations-clean.csv')

In [9]:
df['IoU'] = IoU

In [10]:
df.to_csv(output_path + 'IoU.csv', index = False)